# 安装相关依赖

In [9]:
pip install langchain

In [5]:
pip install langchain-openai

In [7]:
pip show langchain

Name: langchain
Version: 0.2.1
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [6]:
pip show langchain-openai

Name: langchain-openai
Version: 0.1.7
Summary: An integration package connecting OpenAI and LangChain
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: langchain-core, openai, tiktoken
Required-by: 


# 配置全局环境变量

In [10]:
import os
os.environ["OPENAI_API_KEY"] = "*****"
os.environ["OPENAI_API_BASE"] = "https://******"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "*****"

# 简单对话

In [14]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo")

In [15]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hi Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-23a82a34-f184-4995-b94c-dbb698e29688-0')

In [16]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, I do not have access to that information.", response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 12, 'total_tokens': 25}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2a9a4b2e-686a-4c53-bcac-506afad2e2e3-0')

携带历史消息

In [17]:
from langchain_core.messages import AIMessage
model.invoke(
[
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
]
)

AIMessage(content='Your name is Bob. How can I assist you, Bob?', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 35, 'total_tokens': 48}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0d0ed529-4bc5-4a38-8c04-d03e2471214a-0')

In [23]:
pip install langchain_community

自动携带历史对话

In [24]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
	if session_id not in store:
	        store[session_id] = ChatMessageHistory()
	return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [30]:
config = {"configurable": {"session_id": "abdd2"}}

In [31]:
response = with_message_history.invoke(
[HumanMessage(content="Hi! I'm Bob")],
    config=config,
)
response.content

'Hi Bob, nice to meet you! How can I assist you today?'

In [32]:
response = with_message_history.invoke(
[HumanMessage(content="What's my name?")],
    config=config,
)
response.content

'Your name is Bob.'

In [33]:
config = {"configurable": {"session_id": "abc3"}}
response = with_message_history.invoke(
[HumanMessage(content="What's my name?")],
    config=config,
)
response.content

"I'm sorry, I do not have access to personal information such as your name."

In [34]:
config = {"configurable": {"session_id": "abdd2"}}
response = with_message_history.invoke(
[HumanMessage(content="What's my name?")],
    config=config,
)
response.content

'Your name is Bob.'

## 加入提示词模板

In [35]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
[
(
"system",
"You are a helpful assistant. Answer all questions to the best of your ability.",
),
        MessagesPlaceholder(variable_name="messages"),
]
)
chain = prompt | model

In [37]:
response = chain.invoke({"messages": [HumanMessage(content="你好，我是小蓝")]})
response.content

'你好，小蓝！有什么可以帮助你的吗？'

定义`RunnableWithMessageHistory`

In [38]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [42]:
config = {"configurable": {"session_id": "abc2"}}

In [43]:
response = with_message_history.invoke(
[HumanMessage(content="my name is xiaohui")],
    config=config,
)
response.content

'Nice to meet you, Xiaohui! How can I help you today?'

In [44]:
response = with_message_history.invoke(
[HumanMessage(content="who i am?")],
    config=config,
)
response.content

'You are Xiaohui. You are currently interacting with me, a virtual assistant. How can I assist you further, Xiaohui?'

## 提示词模板添加变量

In [45]:
prompt = ChatPromptTemplate.from_messages(
[
(
"system",
"You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
),
        MessagesPlaceholder(variable_name="messages"),
]
)
chain = prompt | model

In [46]:
response = chain.invoke(
{"messages": [HumanMessage(content="hi! I'm bob")], "language": "chinese"}
)
response.content

'你好，Bob！有什么我可以帮到你的吗？'

定义`RunnableWithMessageHistory`

In [47]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [48]:
config = {"configurable": {"session_id": "abc3"}}

In [49]:
response = with_message_history.invoke(
{"messages": [HumanMessage(content="hi! I'm todd")], "language": "chinese"},
    config=config,
)
response.content

'你好，Todd！有什么我可以帮助你的吗？'

In [50]:
response = with_message_history.invoke(
{"messages": [HumanMessage(content="whats my name?")], "language": "chinese"},
    config=config,
)
response.content

'你的名字是Todd。有什么我可以帮助你的吗？'

# 管理对话历史

In [64]:
from langchain_core.runnables import RunnablePassthrough
def filter_messages(messages, k=4):
	return messages[-k:]
chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
| prompt
| model
)

In [65]:
messages = [
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
]

In [69]:
response = chain.invoke(
{
"messages": messages + [HumanMessage(content="what's my name?")],
"language": "Chinese",
}
)
response.content

"I'm sorry, but I don't have access to your personal information."

In [70]:
response = chain.invoke(
{
"messages": messages + [HumanMessage(content="what's my fav ice cream")],
"language": "Chinese",
}
)
response.content

'你喜欢香草口味的冰淇淋。'

历史对话

In [90]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable": {"session_id": "abc7"}}

In [91]:
response = with_message_history.invoke(
{
"messages": [HumanMessage(content="whats my name?")],
"language": "Chinese",
},
    config=config,
)
response.content

'抱歉，我无法知道您的名字，因为我是一个虚拟助手。有什么我可以帮到您的吗？'

In [92]:
response = with_message_history.invoke(
{
"messages": [HumanMessage(content="my name is xiaohui")],
"language": "Chinese",
},
    config=config,
)
response.content

'您好，Xiaohui先生/女士！有什么问题我可以帮到您的吗？'

In [93]:
response = with_message_history.invoke(
{
"messages": [HumanMessage(content="whats my name?")],
"language": "Chinese",
},
    config=config,
)
response.content

'您的名字是Xiaohui。有什么其他问题我可以帮到您的吗？'

In [94]:
response = with_message_history.invoke(
{
"messages": [HumanMessage(content="1+1=")],
"language": "Chinese",
},
    config=config,
)
response.content

'1 + 1 = 2。有其他问题需要我帮忙解答吗？'

In [95]:
response = with_message_history.invoke(
{
"messages": [HumanMessage(content="2+2=")],
"language": "Chinese",
},
    config=config,
)
response.content

'2 + 2 = 4。还有其他问题吗？'

In [96]:
response = with_message_history.invoke(
{
"messages": [HumanMessage(content="3+3=")],
"language": "Chinese",
},
    config=config,
)
response.content

'3 + 3 = 6。还有什么需要帮忙的吗？'

In [97]:
response = with_message_history.invoke(
{
"messages": [HumanMessage(content="whats my name?")],
"language": "Chinese",
},
    config=config,
)
response.content

'对不起，我无法知道您的名字。有其他问题需要我回答吗？'

In [98]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
{
"messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
"language": "Chinese",
},
    config=config,
):
  print(r.content, end="|")

|你|好|，|Todd|！|这|是|一个|笑|话|：|为|什|么|秋|天|是|派|对|季|节|？|因|为|树|叶|在|掉|落|！|希|望|你|喜|欢|这|个|笑|话|！|有|什|么|其他|问题|我|可以|帮|你|解|答|吗|？||